# Load data

In [3]:
import os
import sys
import argparse
import pandas as pd
import re
import numpy as np


In [4]:
snp_info = pd.read_excel("/Users/petterbystrom/Documents/Applied Bioinformatics/Python/Benchmarking_ASR_algorithms/snp_spreadsheet.xlsx", "Supp. table 14", header=1, index_col=0)
taxa_info = pd.read_excel("/Users/petterbystrom/Documents/Applied Bioinformatics/Python/Benchmarking_ASR_algorithms/snp_spreadsheet.xlsx", "Supp. table 13", header=1)
headers_path="/Users/petterbystrom/Documents/Applied Bioinformatics/Headers.txt"
annotation_file = "/Users/petterbystrom/Documents/Applied Bioinformatics/Python/genomic.gtf"

In [4]:
os.getcwd()

'/Users/petterbystrom/Documents/Applied Bioinformatics/Python/Benchmarking_ASR_algorithms'

In [ ]:
/Users/petterbystrom/Documents/Applied\ Bioinformatics/Python/Benchmarking_ASR_algorithms/snp_spreadsheet.xlsx
/Users/petterbystrom/Documents/Applied\ Bioinformatics/Python/Benchmarking_ASR_algorithms/genomic.gtf
/Users/petterbystrom/Documents/Applied\ Bioinformatics/snpAlignment_Spyrou2022.fasta
/Users/petterbystrom/Documents/Applied\ Bioinformatics/


In [ ]:
python snp_to_codon_alignment.py --snp_align_path /Users/petterbystrom/Documents/Applied\ Bioinformatics/snpAlignment_Spyrou2022.fasta --full_align_path /Users/petterbystrom/Documents/Applied\ Bioinformatics/snpAlignment_Spyrou2022.fasta  --snp_info_path /Users/petterbystrom/Documents/Applied\ Bioinformatics/Python/Benchmarking_ASR_algorithms/snp_spreadsheet.xlsx --annotation_path /Users/petterbystrom/Documents/Applied\ Bioinformatics/Python/Benchmarking_ASR_algorithms/genomic.gtf --outdir /Users/petterbystrom/Documents/Applied\ Bioinformatics/

### open Headers.txt and make into list without > and :

In [5]:
fasta = "test.fasta"
def headers_to_list(fasta_file):
    with open(fasta_file, 'r') as file:
        file_contents = file.readlines()
    return sorted([line.replace('>', '').replace(':', '').replace('\n','') for line in file_contents])
headers = headers_to_list("/Users/petterbystrom/Documents/Applied Bioinformatics/snpAlignment_Spyrou2022.fasta")

In [6]:
def headers_from_fasta(fasta_file):
    headers = []
    
    with open(fasta_file, 'r') as fasta_file:
        current_header = None
        for line in fasta_file:
            line = line.strip()
            if line.startswith('>'):
                # If a line starts with '>', it's a header line
                current_header = line
                headers.append(current_header)
    
    return [line.replace('>', '') for line in headers]

In [7]:
print(headers_from_fasta(fasta))

['Reference_gi|16120353|ref|NC_003143.1| Yersinia pestis CO92 chromosome, complete genome', 'Azov38']


### Make set of common elements

In [8]:
def create_regex(prefix):
    return re.compile(fr'^{prefix}(?:\.[A-Za-z0-9]+)?$')

In [9]:
def transform_elements(regex, elements):
    pattern = re.compile(regex)
    transformed_elements = []
    for element in elements:
        match = pattern.match(element)
        if match:
            transformed_elements.append(re.sub(r'\.[A-Za-z0-9]+$', '', element))
        else:
            transformed_elements.append(element)
    return transformed_elements
regex_pattern = r'^[A-Za-z0-9]+(?:\.[A-Za-z0-9]+)?$' # Your regex pattern
'''
# Test data
my_list = ['BED024', 'BED024.A0102', 'BED024.B0234', 'ELW098', 'ELW098.B0421', 'BED024.', 'BED024A0102']


transformed_list = transform_elements(regex_pattern, Headers_from_fasta[200:])
print(Headers_from_fasta[200:])
print("Transformed list:", transformed_list)
'''

'\n# Test data\nmy_list = [\'BED024\', \'BED024.A0102\', \'BED024.B0234\', \'ELW098\', \'ELW098.B0421\', \'BED024.\', \'BED024A0102\']\n\n\ntransformed_list = transform_elements(regex_pattern, Headers_from_fasta[200:])\nprint(Headers_from_fasta[200:])\nprint("Transformed list:", transformed_list)\n'

In [10]:
#enteries = set(snp_info.columns.to_list()) & set(taxa_info["Genome ID"].to_list()) & set(transform_elements(regex_pattern,headers))
enteries = set(snp_info.columns.to_list()) & set(taxa_info["Genome ID"].to_list()) & set(headers)
print(len(enteries))
unique_to_snpinfo = [element for element in snp_info.columns.to_list() if element not in enteries]
unique_to_taxainfo = [element for element in set(taxa_info["Genome ID"].to_list()) if element not in enteries]
unique_to_Headersfromfasta = [element for element in transform_elements(regex_pattern,headers) if element not in enteries]
print(sorted(unique_to_snpinfo))
print(sorted(unique_to_taxainfo))
print(sorted(unique_to_Headersfromfasta))
print(enteries)

218
['AGU007', 'AGU010', 'AGU025', 'BED024', 'BED028', 'BED030', 'BED034', 'BRA001', 'BSK001', 'BSK001-003', 'BSK003', 'Barcelona3031', 'Bolgar2370', 'CHE001', 'CO92 Ref.', 'COL1', 'ELW098', 'LAI009', 'LBG002', 'London-6330', 'London-ES', 'MAN008', 'NAB003', 'NMS002', 'SC2-SPN7', 'STA001', 'STN002', 'STN007', 'STN008', 'STN013', 'STN014', 'STN019', 'STN020', 'STN021', 'TRP002', 'Y.pseudotuberculosis IP32953 (outgroup)']
['AGU007', 'AGU010', 'AGU025', 'BED024', 'BED028', 'BED030', 'BED034', 'BRA001', 'Barcelona 3031', 'Bolgar2370', 'CHE1', 'COL1', 'ELW098', 'LAI009', 'LBG002', 'London  6330', 'London 8124/8291/11972', 'MAN008', 'NAB003', 'NMS002', 'SC2_SPN7', 'STA001', 'STN002', 'STN007', 'STN008', 'STN013', 'STN014', 'STN019', 'STN020', 'STN021', 'TRP002']
['...', 'AGU007', 'AGU010', 'AGU025', 'ANAGGCCAACGCTGCCAGGCTGGGGNNNCGNNANCCTCTNTCGCTAGGNGGTNCNGNGCG', 'ANAGGGCGCCAATGAAATACGTCCTCACCGGCCCATACACTGAGTGATTGGGCCGGACCC', 'ANAGGNNNCCAATGAAATACGTNCTNANNGNNNCATANNNTGAGTGATTGGNCCGGANCC', 'AN

In [42]:
os.path.basename("genomic.gtf")

'genomic.gtf'

In [11]:
#make fasta file of all matching enteries
def extract_sequences_by_headers(input_file, output_file, headers):
    sequences = {}
    header = ''
    with open(input_file, 'r') as file:
        for line in file:
            if line.startswith('>'):
                line = line.replace('>', '')
                print(line)
                header = line.strip()
                sequences[header] = ''
            else:
                sequences[header] += line.strip()

    with open(output_file, 'w') as out_file:
        for h in headers:
            if h in sequences:
                out_file.write(f'>{h}\n{sequences[h]}\n')

# List of headers to extract
headers_to_extract = ['>Header1', '>Header2', '>Header3']  # Replace this with your list of headers

# Paths to input and output files
input_fasta = "/Users/petterbystrom/Documents/Applied Bioinformatics/snpAlignment_Spyrou2022.fasta"  # Replace with your input FASTA file
output_fasta = 'output.fasta'  # Replace with the desired output FASTA file

# Extract and write sequences based on the headers list
extract_sequences_by_headers(input_fasta, output_fasta, list(enteries))
print(list(enteries))


BSK001-003.A0101.A0102.A0103-malt

Azov38

Rostov2033

Gdansk8

G701

COLC1-COLC2a_COLC2b

Rostov2039

SC2_SPN7

CHE.C1-C2-3-C4

PEB10

G488

AGU025.B0102

AGU007.B0102

AGU010.B0102

OSL1

Ber37

Ber45

NMS002.A0101

BED024.A0102

BED028.A0102

BED030.A0102

BED034.A0102

BRA001.A0101

ELW098_JK1548

LAI009.A0101

LBG002.A0101

MAN008.B0101

NAB003.B0101

STA001.A0101

STN002.A0101

STN007.A0101

STN013.A0101

STN014.A0101

STN020.A0101

STN021.A0101

STN008.A0101

STN019.A0101

TRP002.A0101-02

London_StMaryGraces_6330

London_EastSmithfield_8124_8291_11972

Bolgar

Barcelona

OBS124

OBS107

OBS116

OBS137

OBS110

0.ANT2_B42003004

0.PE2_PEST-F

0.PE4Ba_PestoidesA

1.ANT1_UG05-0454

1.ORI1_CA88

1.ORI1_CO92

1.ORI1b_India195

1.ORI2_F1991016

1.ORI3_IP275

1.ORI3_MG05-1020

2.ANT1_Nepal516

2.MED1_K1973002

2.MED1_KIM10

outgroup_Y.pseudo

0.ANT1a_42013

0.ANT1b_CMCC49003

0.ANT1c_945

0.ANT1d_164

0.ANT1e_CMCC8211

0.ANT1f_42095

0.ANT1g_CMCC42007

0.ANT1h_CMCC43032

0.ANT2a_2330


### Make translation dictionary

In [ ]:
modern_ancient_dict={}

### Make dictionary of Genome ID and coresponding decent

In [48]:
lineage_to_modern_ancient = dict(zip(taxa_info['Genome ID'], taxa_info['Modern/Ancient']))

### Find amount of ambigious sites in SNP info

In [49]:
rows_without_N = snp_info[~snp_info.apply(lambda row: row.astype(str).str.contains('N', case=False)).any(axis=1)]
print(rows_without_N)

         CO92 Ref. BSK001 BSK003 BSK001-003 LAI009 TRP002 London-ES  \
Position                                                              
37356            G      .      .          .      .      .         .   
41758            G      .      .          .      .      .         .   
63229            C      .      .          .      .      .         .   
75934            C      .      .          .      .      .         .   
78073            G      .      .          .      .      .         .   
...            ...    ...    ...        ...    ...    ...       ...   
4595001          C      .      .          .      .      .         .   
4600959          G      .      .          .      .      .         .   
4625210          G      .      .          .      .      .         .   
4630765          G      .      .          .      .      .         .   
4634211          C      .      .          .      .      .         .   

         Barcelona3031 OSL1 NAB003  ... 4.ANT_1454 4.ANT_338 4.ANT_517  \
Po

### Extract information from annotation file

In [12]:
with open("genomic.gtf", 'r') as file:
    file_contents = file.readlines()

In [13]:
Annotation_dict={}
for i in file_contents:
    match_geneid = re.search(r'gene_id "([A-Za-z0-9_]+)"',i)
    match_location = re.search(r'\tgene\t(\d+)\t(\d+)\t',i)
    match_direction = re.search(r'(?<=\t\.\t)([\-\+])(?=\t\.)',i)
    if match_geneid and match_location and match_direction:
        Annotation_dict[match_geneid.group(1)] = [int(match_location.group(1)),int(match_location.group(2)),match_direction.group(1),0,0]
    
print(Annotation_dict)
print(len(Annotation_dict))
print((len(file_contents)-6)/4)

{'YPO_RS01005': [271, 711, '-', 0, 0], 'YPO_RS01010': [804, 1265, '-', 0, 0], 'YPO_RS01015': [1435, 2427, '+', 0, 0], 'YPO_RS01020': [2526, 3992, '-', 0, 0], 'YPO_RS01025': [3996, 5549, '-', 0, 0], 'YPO_RS01030': [5823, 7691, '+', 0, 0], 'YPO_RS01035': [7896, 8315, '+', 0, 0], 'YPO_RS01040': [8366, 9292, '+', 0, 0], 'YPO_RS01050': [9345, 9515, '+', 0, 0], 'YPO_RS01055': [9512, 10936, '-', 0, 0], 'YPO_RS01060': [11016, 11705, '-', 0, 0], 'YPO_RS01070': [12265, 13807, '+', 0, 0], 'YPO_RS01075': [13944, 14019, '+', 0, 0], 'YPO_RS01080': [14272, 17178, '+', 0, 0], 'YPO_RS01085': [17288, 17403, '+', 0, 0], 'YPO_RS01090': [17906, 19114, '+', 0, 0], 'YPO_RS01095': [19172, 19702, '-', 0, 0], 'YPO_RS01100': [19702, 20289, '-', 0, 0], 'YPO_RS01105': [20443, 20712, '+', 0, 0], 'YPO_RS01110': [20792, 21778, '+', 0, 0], 'YPO_RS01115': [21806, 22429, '+', 0, 0], 'YPO_RS01120': [22921, 25719, '+', 0, 0], 'YPO_RS01125': [26128, 26586, '+', 0, 0], 'YPO_RS01130': [26838, 27488, '-', 0, 0], 'YPO_RS01135'

In [14]:
from Bio import SeqIO
def get_sequence_lengths(fasta_file,header):
    for record in SeqIO.parse(fasta_file, "fasta"):
        #print(record.id)
        seq_len = len(record.seq)
        #print(header[0:len(record.id)])
        #if record.id == header[0:len(record.id)]:
        #    seq_len = len(record.seq)
    return seq_len

In [55]:
print(get_sequence_lengths("/Users/petterbystrom/Documents/Applied Bioinformatics/sequence9.fasta",headers))

Reference_gi|16120353|ref|NC_003143.1|
4653728


In [130]:
print(get_sequence_lengths("test.fasta","Reference_gi|16120353|ref|NC_003143.1| Yersinia pestis CO92 chromosome, complete genome"))

3533


### Make dictioray of gene location and open reading frame

In [40]:
def get_ORF(Annotation_dict):
    for key in Annotation_dict:
        if Annotation_dict[key][2] == "+":
            orf = (Annotation_dict[key][0]-1) % 3
            Annotation_dict[key][3]=orf
            
        elif Annotation_dict[key][2] == "-":
            orf = -1*(((get_sequence_lengths("test.fasta","Reference_gi|16120353|ref|NC_003143.1| Yersinia pestis CO92 chromosome, complete genome")-Annotation_dict[key][1]))%3)
            Annotation_dict[key][3]=orf
    return Annotation_dict



In [41]:
Annotation_dict = get_ORF(Annotation_dict)


In [42]:
print(Annotation_dict)

{'YPO_RS01005': [271, 711, '-', -2, []], 'YPO_RS01010': [804, 1265, '-', 0, [855, 1166]], 'YPO_RS01015': [1435, 2427, '+', 0, [1656, 2052]], 'YPO_RS01020': [2526, 3992, '-', 0, [2873, 3270]], 'YPO_RS01025': [3996, 5549, '-', 0, [4089, 4225]], 'YPO_RS01030': [5823, 7691, '+', 2, [6103]], 'YPO_RS01035': [7896, 8315, '+', 2, []], 'YPO_RS01040': [8366, 9292, '+', 1, [8793]], 'YPO_RS01050': [9345, 9515, '+', 2, []], 'YPO_RS01055': [9512, 10936, '-', -1, []], 'YPO_RS01060': [11016, 11705, '-', 0, [11330]], 'YPO_RS01070': [12265, 13807, '+', 0, []], 'YPO_RS01075': [13944, 14019, '+', 2, []], 'YPO_RS01080': [14272, 17178, '+', 0, []], 'YPO_RS01085': [17288, 17403, '+', 1, []], 'YPO_RS01090': [17906, 19114, '+', 1, []], 'YPO_RS01095': [19172, 19702, '-', -1, []], 'YPO_RS01100': [19702, 20289, '-', -2, [19828, 20252]], 'YPO_RS01105': [20443, 20712, '+', 0, []], 'YPO_RS01110': [20792, 21778, '+', 1, [21219]], 'YPO_RS01115': [21806, 22429, '+', 1, []], 'YPO_RS01120': [22921, 25719, '+', 0, [22948,

Slower solution

In [137]:
def codon_finder(alignment_file, snp_df, Annotation_dict):
    temp_dict = Annotation_dict.copy()
    for key in Annotation_dict:
        temp_dict[key][4] = []
        snp_search =(snp_df.index >= Annotation_dict[key][0]) & (snp_df.index <= Annotation_dict[key][1])
        for i,j in enumerate(snp_search):

            if j==True:
                temp_dict[key][4].append (snp_df.index[i])
            
    return temp_dict

Faster solution

In [43]:
def codon_finder2(alignment_file, snp_df, Annotation_dict):
    for key, value in Annotation_dict.items():
        snp_indices = snp_df[snp_df.index.to_series().between(value[0], value[1])].index.tolist()
        value[4] = snp_indices
    return Annotation_dict

In [172]:
temp_dict = codon_finder("test.fasta",snp_info["Azov38"].to_frame(),Annotation_dict)

print(temp_dict)



{'YPO_RS01005': [271, 711, '-', -3, []], 'YPO_RS01010': [804, 1265, '-', -1, [855, 1166]], 'YPO_RS01015': [1435, 2427, '+', 3, [1656, 2052]], 'YPO_RS01020': [2526, 3992, '-', -1, [2873, 3270]], 'YPO_RS01025': [3996, 5549, '-', -1, [4089, 4225]], 'YPO_RS01030': [5823, 7691, '+', 2, [6103]], 'YPO_RS01035': [7896, 8315, '+', 2, []], 'YPO_RS01040': [8366, 9292, '+', 1, [8793]], 'YPO_RS01050': [9345, 9515, '+', 2, []], 'YPO_RS01055': [9512, 10936, '-', -2, []], 'YPO_RS01060': [11016, 11705, '-', -1, [11330]], 'YPO_RS01070': [12265, 13807, '+', 3, []], 'YPO_RS01075': [13944, 14019, '+', 2, []], 'YPO_RS01080': [14272, 17178, '+', 3, []], 'YPO_RS01085': [17288, 17403, '+', 1, []], 'YPO_RS01090': [17906, 19114, '+', 1, []], 'YPO_RS01095': [19172, 19702, '-', -2, []], 'YPO_RS01100': [19702, 20289, '-', -3, [19828, 20252]], 'YPO_RS01105': [20443, 20712, '+', 3, []], 'YPO_RS01110': [20792, 21778, '+', 1, [21219]], 'YPO_RS01115': [21806, 22429, '+', 1, []], 'YPO_RS01120': [22921, 25719, '+', 3, [22

In [44]:
Annotation_dict = codon_finder2("test.fasta",snp_info["Azov38"].to_frame(),Annotation_dict)

print(Annotation_dict)

{'YPO_RS01005': [271, 711, '-', -2, []], 'YPO_RS01010': [804, 1265, '-', 0, [855, 1166]], 'YPO_RS01015': [1435, 2427, '+', 0, [1656, 2052]], 'YPO_RS01020': [2526, 3992, '-', 0, [2873, 3270]], 'YPO_RS01025': [3996, 5549, '-', 0, [4089, 4225]], 'YPO_RS01030': [5823, 7691, '+', 2, [6103]], 'YPO_RS01035': [7896, 8315, '+', 2, []], 'YPO_RS01040': [8366, 9292, '+', 1, [8793]], 'YPO_RS01050': [9345, 9515, '+', 2, []], 'YPO_RS01055': [9512, 10936, '-', -1, []], 'YPO_RS01060': [11016, 11705, '-', 0, [11330]], 'YPO_RS01070': [12265, 13807, '+', 0, []], 'YPO_RS01075': [13944, 14019, '+', 2, []], 'YPO_RS01080': [14272, 17178, '+', 0, []], 'YPO_RS01085': [17288, 17403, '+', 1, []], 'YPO_RS01090': [17906, 19114, '+', 1, []], 'YPO_RS01095': [19172, 19702, '-', -1, []], 'YPO_RS01100': [19702, 20289, '-', -2, [19828, 20252]], 'YPO_RS01105': [20443, 20712, '+', 0, []], 'YPO_RS01110': [20792, 21778, '+', 1, [21219]], 'YPO_RS01115': [21806, 22429, '+', 1, []], 'YPO_RS01120': [22921, 25719, '+', 0, [22948,

In [234]:
orf = (1656-1)%3
codonpos =(1656-1435)%3
print(orf,codonpos)

2 2


Find bases in codon

In [20]:
def complement_base(base):
    complement_dict = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    return complement_dict.get(base, base)

print(complement_base("C"))

G


In [21]:
def reverse_complement(sequence):
    complement_sequence = [complement_base(base) for base in sequence]
    return ''.join(complement_sequence)[::-1]  # Reversing the sequence

In [53]:
'''
def find_bases(Annotation_dict, fasta_file):
    codons_forward=[]
    codons_reverse =[]
    for key,value in Annotation_dict.items():
        position = 0
        if len(value[-1])>0:
            # + reading frame
            if value[2] == "+":
                for i in value[-1]:
                    position_in_codon = (i-value[0])%3
                    if position_in_codon ==0:
                        codons_forward.append([i,i+1,i+2,value[2]])
                    if position_in_codon ==1:
                        codons_forward.append([i-1,i,i+1,value[2]])
                    if position_in_codon ==2:
                        codons_forward.append([i-2,i-1,i,value[2]])
            if value[2] == "-":
                for i in value[-1]:
                    position_in_codon = (i-value[1])%3
                    if position_in_codon ==0:
                        codons_reverse.append([i,i+1,i+2,value[2]])
                    if position_in_codon ==1:
                        codons_reverse.append([i-1,i,i+1,value[2]])
                    if position_in_codon ==2:
                        codons_reverse.append([i-2,i-1,i,value[2]])
    
    return [codons_forward,codons_reverse]
'''
def find_bases(Annotation_dict):
    codons=[]
    for key,value in Annotation_dict.items():
        if len(value[-1])>0:
            # forward reading frame
            if value[2] == "+":
                for i in value[-1]:
                    position_in_codon = (i-value[0])%3
                    if position_in_codon ==0:
                        codons.append([i,i+1,i+2,position_in_codon,value[2]])
                    if position_in_codon ==1:
                        codons.append([i-1,i,i+1,position_in_codon,value[2]])
                    if position_in_codon ==2:
                        codons.append([i-2,i-1,i,position_in_codon,value[2]])

            # reverse reading frame
            if value[2] == "-":
                for i in value[-1]:
                    position_in_codon = (value[1]-i)%3
                    if position_in_codon ==0:
                        codons.append([i-2,i-1,i,position_in_codon,value[2]])
                    if position_in_codon ==1:
                        codons.append([i-1,i,i+1,position_in_codon,value[2]])
                    if position_in_codon ==2:
                        codons.append([i,i+1,i+2,position_in_codon,value[2]])
    return codons

In [54]:
codons = find_bases(Annotation_dict,"test.fasta")
print(codons)
print((8366-1)%3)
print((8793-8366)%3)



[[855, 856, 857, 2, '-'], [1164, 1165, 1166, 0, '-'], [1654, 1655, 1656, 2, '+'], [2050, 2051, 2052, 2, '+'], [2871, 2872, 2873, 0, '-'], [3270, 3271, 3272, 2, '-'], [4089, 4090, 4091, 2, '-'], [4224, 4225, 4226, 1, '-'], [6102, 6103, 6104, 1, '+'], [8792, 8793, 8794, 1, '+'], [11328, 11329, 11330, 0, '-'], [19828, 19829, 19830, 2, '-'], [20251, 20252, 20253, 1, '-'], [21218, 21219, 21220, 1, '+'], [22948, 22949, 22950, 0, '+'], [25459, 25460, 25461, 2, '+'], [29366, 29367, 29368, 2, '+'], [30883, 30884, 30885, 1, '-'], [32156, 32157, 32158, 2, '-'], [32165, 32166, 32167, 1, '-'], [36182, 36183, 36184, 0, '+'], [37355, 37356, 37357, 1, '+'], [41536, 41537, 41538, 0, '+'], [41758, 41759, 41760, 0, '+'], [42301, 42302, 42303, 2, '+'], [43341, 43342, 43343, 0, '-'], [45564, 45565, 45566, 0, '-'], [47957, 47958, 47959, 2, '-'], [48866, 48867, 48868, 1, '-'], [50419, 50420, 50421, 1, '-'], [50875, 50876, 50877, 0, '-'], [52502, 52503, 52504, 2, '-'], [52502, 52503, 52504, 1, '-'], [52529, 5

In [272]:
print(codons)

[[853, 854, 855, -1], [1166, 1167, 1168, -1], [1654, 1655, 1656, 2], [2050, 2051, 2052, 2], [2873, 2874, 2875, -1], [3268, 3269, 3270, -1], [4087, 4088, 4089, -1], [4224, 4225, 4226, -1], [6102, 6103, 6104, 1], [8792, 8793, 8794, 1], [11330, 11331, 11332, -1], [19826, 19827, 19828, -3], [20251, 20252, 20253, -3], [21218, 21219, 21220, 1], [22948, 22949, 22950, 0], [25459, 25460, 25461, 2], [29366, 29367, 29368, 2], [30883, 30884, 30885, -3], [32154, 32155, 32156, -2], [32165, 32166, 32167, -2], [36182, 36183, 36184, 0], [37355, 37356, 37357, 1], [41536, 41537, 41538, 0], [41758, 41759, 41760, 0], [42301, 42302, 42303, 2], [43343, 43344, 43345, -1], [45566, 45567, 45568, -1], [47955, 47956, 47957, -2], [48866, 48867, 48868, -2], [50419, 50420, 50421, -3], [50877, 50878, 50879, -3], [52500, 52501, 52502, -2], [52502, 52503, 52504, -2], [52531, 52532, 52533, -2], [53010, 53011, 53012, -2], [54102, 54103, 54104, 0], [54672, 54673, 54674, 2], [58822, 58823, 58824, 2], [58828, 58829, 58830, 

In [32]:
s='abc'
s+='d'
print(s)
#print(snp_info.head(20))
snp_info = snp_info.apply(lambda col: np.where(col == '.', snp_info['CO92 Ref.'], col))
#df = snp_info.mask(df == '.', df['CO92 Ref.'], axis=0)
pd.set_option('display.max_rows', 100)
display(snp_info.head(30))

abcd


,CO92 Ref.,BSK001,BSK003,BSK001-003,LAI009,TRP002,London-ES,Barcelona3031,OSL1,NAB003,...,4.ANT_1454,4.ANT_338,4.ANT_517,4.ANT_KM932,4.ANT_M-1944,0.PE5_I-2236,0.PE5_I-3190,0.PE5_I-2231,0.PE5_I-2457,Y.pseudotuberculosis IP32953 (outgroup)
Position,,,,,,,,,,,,,,,,,,,,,
17,C,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
855,G,G,G,G,G,G,G,G,G,G,...,G,G,G,G,G,G,G,G,G,G
1166,C,C,C,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
1315,C,N,N,N,N,N,N,C,C,N,...,C,C,C,C,C,C,C,C,C,C
1333,G,G,N,G,G,N,N,G,G,N,...,G,G,G,G,G,A,A,A,A,G
1656,G,G,N,G,G,G,G,G,G,G,...,G,G,G,G,G,G,G,G,G,G
2052,A,A,A,A,A,A,A,A,A,A,...,A,A,A,A,A,A,A,A,A,A
2873,G,G,G,G,G,N,G,G,G,G,...,G,G,G,G,G,G,G,G,G,G
3270,C,C,N,C,C,C,C,C,C,C,...,C,C,C,C,C,T,T,T,T,C


In [33]:
print(Annotation_dict)

{'YPO_RS01005': [271, 711, '-', -3, []], 'YPO_RS01010': [804, 1265, '-', -1, [855, 1166]], 'YPO_RS01015': [1435, 2427, '+', 0, [1656, 2052]], 'YPO_RS01020': [2526, 3992, '-', -1, [2873, 3270]], 'YPO_RS01025': [3996, 5549, '-', -1, [4089, 4225]], 'YPO_RS01030': [5823, 7691, '+', 2, [6103]], 'YPO_RS01035': [7896, 8315, '+', 2, []], 'YPO_RS01040': [8366, 9292, '+', 1, [8793]], 'YPO_RS01050': [9345, 9515, '+', 2, []], 'YPO_RS01055': [9512, 10936, '-', -2, []], 'YPO_RS01060': [11016, 11705, '-', -1, [11330]], 'YPO_RS01070': [12265, 13807, '+', 0, []], 'YPO_RS01075': [13944, 14019, '+', 2, []], 'YPO_RS01080': [14272, 17178, '+', 0, []], 'YPO_RS01085': [17288, 17403, '+', 1, []], 'YPO_RS01090': [17906, 19114, '+', 1, []], 'YPO_RS01095': [19172, 19702, '-', -2, []], 'YPO_RS01100': [19702, 20289, '-', -3, [19828, 20252]], 'YPO_RS01105': [20443, 20712, '+', 0, []], 'YPO_RS01110': [20792, 21778, '+', 1, [21219]], 'YPO_RS01115': [21806, 22429, '+', 1, []], 'YPO_RS01120': [22921, 25719, '+', 0, [22

In [56]:
print((8366-1)%3)
print((8793-8366)%3)

print((1265-855)%3)

1
1
2


In [90]:
print(len(codons))

2790


In [88]:
for i in SeqIO.parse("/Users/petterbystrom/Documents/Applied Bioinformatics/sequence9.fasta", 'fasta'):
    sequence=str(i.seq)
snp_align_sequence = SeqIO.parse("/Users/petterbystrom/Documents/Applied Bioinformatics/snpAlignment_Spyrou2022.fasta", 'fasta')
extracted_sequences={}
locations = [[854, 855, 856], [1166, 1167, 1168]]
#for seq_record in sequences:
#    header = seq_record.id
for seq_record in snp_align_sequence:# iterate throuh snp
    header = seq_record.id 
    #if header in list(snp_info.columns.to_list()):
    #sequence = str(seq_record.seq)
    if header in list(snp_info.columns.to_list()):
        extracted_sequence=""
        for i in codons:
            if i[4]=='+':
                if i[3]==0:
                    extracted_sequence+=snp_info.loc[i[0],header]+sequence[i[1]] +sequence[i[2]]
                elif i[3]==1:
                    extracted_sequence+=sequence[i[0]] + snp_info.loc[i[1],header]+sequence[i[2]]
                elif i[3]==2:  
                    extracted_sequence+=sequence[i[0]] +sequence[i[1]]+ snp_info.loc[i[2],header]
            elif i[4]=='-':
                if i[3]==0:
                    extracted_sequence+=reverse_complement(sequence[i[0]] +sequence[i[1]]+ snp_info.loc[i[2],header])
                elif i[3]==1:
                    extracted_sequence+=reverse_complement(sequence[i[0]] + snp_info.loc[i[1],header]+sequence[i[2]])
                elif i[3]==2:
                    extracted_sequence+=reverse_complement(snp_info.loc[i[0],header]+sequence[i[1]] +sequence[i[2]])
            extracted_sequences[header]= extracted_sequence
    # Extract sequences based on the provided locations
    #extracted_sequences[header] = ''.join(sequence[i - 1] for loc in codons for i in loc)
# Write the extracted sequences to a new FASTA file
with open("output.fasta", 'w') as output_file:
    for header, sequence in extracted_sequences.items():
        output_file.write(f'>{header}\n{sequence}\n')
        

In [89]:
snp_align_sequence = SeqIO.parse("/Users/petterbystrom/Documents/Applied Bioinformatics/snpAlignment_Spyrou2022.fasta", 'fasta')
print(snp_align_sequence)

for seq_record in snp_align_sequence:# iterate throuh snp
    seq_records=seq_record.seq
print(len(seq_records))

3533


### codon file is shorter than 3x snp file because snp outside cds are deleted